In [ ]:
#Scraping MLB data with BeautifulSoup and Selenium

In [74]:
import os
os.getcwd()

'C:\\Users\\dave_\\Desktop\\git_test\\projects\\clean'

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_MN = pd.read_html('https://en.wikipedia.org/wiki/Minnesota')

In [21]:
print(f'Total tables: {len(table_MN)}')

Total tables: 29


In [24]:
table_MN = pd.read_html('https://en.wikipedia.org/wiki/Minnesota', match='Election results from statewide races')
len(table_MN)

1

In [26]:
df = table_MN[0]
df.head()

,Year,Office,GOP,DFL,Others
0,2020,President,45.3%,52.4%,2.3%
1,2020,Senator,43.5%,48.8%,7.7%
2,2018,Governor,42.4%,53.9%,3.7%
3,2018,Senator,36.2%,60.3%,3.4%
4,2018,Senator,42.4%,53.0%,4.6%


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    26 non-null     int64 
 1   Office  26 non-null     object
 2   GOP     26 non-null     object
 3   DFL     26 non-null     object
 4   Others  26 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.1+ KB


In [34]:
#df['GOP'].replace({'%':''}, regex=True).astype('float')

df = df.replace({'%': ''}, regex=True)
df[['GOP', 'DFL', 'Others']] = df[['GOP', 'DFL', 'Others']].apply(pd.to_numeric)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    26 non-null     int64  
 1   Office  26 non-null     object 
 2   GOP     26 non-null     float64
 3   DFL     26 non-null     float64
 4   Others  26 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.1+ KB


# bs4 

In [36]:
import requests
from bs4 import BeautifulSoup

In [39]:
url = "https://zlliu.s3.ap-southeast-1.amazonaws.com/index.html"
r = requests.get(url)
r

<Response [200]>

In [40]:
r.content

b'<div class="header">\n    <a class="logo">Logo goes here</a>\n\n    <div class="header-right">\n        <a class="active" href="">Home</a>\n        <a href="">Contact</a>\n        <a href="">About</a>\n    </div>\n</div>\n  \n<div class="content">\n\n    <div class="row">\n\n        <div class="left">\n            <div>\n                Shipped from: <span id="origin"> Singapore </span>\n            </div><br>\n            <div id="description">\n                Only the freshest fruits shipped from the state of Singapore, \n                also historically known by its indigenous name Pulau Ujong.\n                Description continues ...\n            </div><br>\n            <div>Last updated: <span id="last-updated-datetime">20 Nov 21, 1500 hours</span></div>\n        </div>\n\n        <div class="right">\n            <h2>Fruits for sale</h2>\n            <table id="fruits">\n                <tr><th>Fruit</th><th>Price ($)</th></tr>\n                <tr><td>Apple</td><td>4</td></

In [47]:
soup = BeautifulSoup(r.content, "html.parser")
soup

<div class="header">
<a class="logo">Logo goes here</a>
<div class="header-right">
<a class="active" href="">Home</a>
<a href="">Contact</a>
<a href="">About</a>
</div>
</div>
<div class="content">
<div class="row">
<div class="left">
<div>
                Shipped from: <span id="origin"> Singapore </span>
</div><br/>
<div id="description">
                Only the freshest fruits shipped from the state of Singapore, 
                also historically known by its indigenous name Pulau Ujong.
                Description continues ...
            </div><br/>
<div>Last updated: <span id="last-updated-datetime">20 Nov 21, 1500 hours</span></div>
</div>
<div class="right">
<h2>Fruits for sale</h2>
<table id="fruits">
<tr><th>Fruit</th><th>Price ($)</th></tr>
<tr><td>Apple</td><td>4</td></tr>
<tr><td>Orange</td><td>5</td></tr>
<tr><td>Pear</td><td>6</td></tr>
<tr><td>Durian</td><td>7</td></tr>
</table>
</div>
</div>
</div>
<div id="other-stuff">
    Other stuff goes here
</div>
<style>

   

In [49]:
origin = soup.find('span', {'id':'origin'})
origin

<span id="origin"> Singapore </span>

In [50]:
print(origin.text)

 Singapore 


In [52]:
last_updated = soup.find("span", {'id': "last-updated-datetime"})
last_updated

<span id="last-updated-datetime">20 Nov 21, 1500 hours</span>

In [53]:
last_updated.text

'20 Nov 21, 1500 hours'

In [54]:
table = soup.find("table", {"id":"fruits"})
table

<table id="fruits">
<tr><th>Fruit</th><th>Price ($)</th></tr>
<tr><td>Apple</td><td>4</td></tr>
<tr><td>Orange</td><td>5</td></tr>
<tr><td>Pear</td><td>6</td></tr>
<tr><td>Durian</td><td>7</td></tr>
</table>

In [60]:
fruits = {}
for tr in table.find_all("tr"):
  tds = tr.find_all("td")
  
  #so we dont get table headers
  if len(tds) == 0:
    continue
    
  fruit = tds[0].text
  price = tds[1].text
  
  fruits[fruit]=price
  
print(fruits)

{'Apple': '4', 'Orange': '5', 'Pear': '6', 'Durian': '7'}


In [61]:
#all the code in one place
import requests
from bs4 import BeautifulSoup

url = "https://zlliu.s3.ap-southeast-1.amazonaws.com/index.html"

r = requests.get(url)

soup = BeautifulSoup(r.content, "html.parser")

origin = soup.find("span", {"id":"origin"})
origin = origin.text.strip()

last_updated = soup.find("span", {"id":"last-updated-datetime"})
last_updated = last_updated.text.strip()

table = soup.find("table", {"id": "fruits"})

fruits = {}
for tr in table.find_all("tr"):
    tds = tr.find_all("td")

    # so we don't get the table headers
    if len(tds) == 0:
        continue

    fruit = tds[0].text
    price = tds[1].text

    fruits[fruit] = price

print(origin)
print(last_updated)
print(fruits)

Singapore
20 Nov 21, 1500 hours
{'Apple': '4', 'Orange': '5', 'Pear': '6', 'Durian': '7'}


# bs4 on mlb

In [79]:
#all the code in one place
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/MLB:_The_Show"

r = requests.get(url)

soup = BeautifulSoup(r.content, "html.parser")

soup
predecessors = soup.find("table", {"class":"wikitable"})
#predecessors = predecessors.text.strip()
predecessors

<table class="wikitable">
<tbody><tr>
<th rowspan="2">Game
</th>
<th rowspan="2">Release date
</th>
<th colspan="2">Cover Athlete
</th>
<th rowspan="2">Platforms
</th></tr>
<tr>
<th>Star
</th>
<th>Team
</th></tr>
<tr>
<td><i><a href="/wiki/MLB_%2798" title="MLB '98">MLB '98</a></i>
</td>
<td>July 1, 1997
</td>
<td><a href="/wiki/Bernie_Williams" title="Bernie Williams">Bernie Williams</a>
</td>
<td><a href="/wiki/New_York_Yankees" title="New York Yankees">New York Yankees</a>
</td>
<td rowspan="6"><a href="/wiki/PlayStation_(console)" title="PlayStation (console)">PlayStation</a>
</td></tr>
<tr>
<td><i><a href="/wiki/MLB_%2799" title="MLB '99">MLB '99</a></i>
</td>
<td>March 31, 1998
</td>
<td><a href="/wiki/Cal_Ripken_Jr." title="Cal Ripken Jr.">Cal Ripken Jr.</a>
</td>
<td><a href="/wiki/Baltimore_Orioles" title="Baltimore Orioles">Baltimore Orioles</a>
</td></tr>
<tr>
<td><i><a href="/wiki/MLB_2000" title="MLB 2000">MLB 2000</a></i>
</td>
<td>February 28, 1999
</td>
<td><a href="/wi

In [135]:
predecessors.find_all('a')

team = soup.find("a", {'href': "/wiki/Los_Angeles_Angels"})
print(team)

<a href="/wiki/Los_Angeles_Angels" title="Los Angeles Angels">Anaheim Angels</a>


In [142]:
fruits = {}
for tr in predecessors.find_all("tr"):
    tds = tr.find_all("td")

    # so we don't get the table headers
    if len(tds) == 0:
        continue

    fruit = tds[0].text
    price = tds[2].text

    fruits[fruit] = price

fruits

{"MLB '98\n": 'Bernie Williams\n',
 "MLB '99\n": 'Cal Ripken Jr.\n',
 'MLB 2000\n': 'Mo Vaughn\n',
 'MLB 2001\n': 'Chipper Jones\n',
 'MLB 2002\n': 'Andruw Jones\n',
 'MLB 2003\n': 'Barry Bonds\n',
 'MLB 2004\n': 'Shawn Green\n',
 'MLB 2005\n': 'Eric Chavez\n',
 'MLB 2006\n': 'Vladimir Guerrero\n'}

In [216]:
f = pd.DataFrame.from_dict(fruits, orient='index')
f.reset_index(inplace=True)
f.columns = ['Game Name', 'Cover Athlete']
#f.iloc[0].apply(lambda x:x.strip('\n'))
f['Game Name'] = f['Game Name'].str.replace('\n','')
f['Cover Athlete'] = f['Cover Athlete'].str.replace('\n','')
f

,Game Name,Cover Athlete
0,MLB '98,Bernie Williams
1,MLB '99,Cal Ripken Jr.
2,MLB 2000,Mo Vaughn
3,MLB 2001,Chipper Jones
4,MLB 2002,Andruw Jones
5,MLB 2003,Barry Bonds
6,MLB 2004,Shawn Green
7,MLB 2005,Eric Chavez
8,MLB 2006,Vladimir Guerrero


In [76]:
lst = pd.read_html('https://en.wikipedia.org/wiki/MLB:_The_Show')
lst

[                MLB: The Show  \
 0  MLB: The Show current logo   
 1                    Genre(s)   
 2                Developer(s)   
 3                Publisher(s)   
 4                 Platform(s)   
 5               First release   
 6              Latest release   
 
                                      MLB: The Show.1  
 0                         MLB: The Show current logo  
 1                                             Sports  
 2                                   San Diego Studio  
 3   Sony Interactive EntertainmentMLB Advanced Media  
 4  PlayStation 2, PlayStation 3, PlayStation Port...  
 5                  MLB 06: The ShowFebruary 28, 2006  
 6                       MLB The Show 22April 5, 2022  ,
        Game       Release date      Cover Athlete                        \
        Game       Release date               Star                  Team   
 0   MLB '98       July 1, 1997    Bernie Williams      New York Yankees   
 1   MLB '99     March 31, 1998     Cal Ripken Jr

# selenium

In [63]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
url = 'https://pubs.rsc.org/en/content/articlelanding/2020/na/d0na00118j'

In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

path = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(path)

driver.get('https://www.baseball-reference.com/players/c/cabremi01.shtml')
print(driver.title)


search = driver.find_element_by_id('standard_fielding')
print(search.text)


driver.quit()

<ipython-input-39-fc2c8cc820c6>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Miguel Cabrera Stats | Baseball-Reference.com


<ipython-input-39-fc2c8cc820c6>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_id('standard_fielding')


Year Age Tm Pos Lg G GS CG Inn Ch PO A E DP Fld% Rtot Rdrs Rtot/yr Rdrs/yr RF/9 RF/G lgFld% lgRF9 lgRFG Awards
2003 20 FLA LF NL 55 55 51 481.0 107 99 5 3 1 .972 -5 1 -12 2 1.95 1.89 .979 1.91 1.89 MVP-27,RoY-5
2003 20 FLA OF NL 55 55 51 481.0 107 99 5 3 1 .972 -5 1 -12 2 1.95 1.89 .984 2.17 2.15 MVP-27,RoY-5
2003 20 FLA 3B NL 34 30 30 275.0 71 17 53 1 2 .986 2 -3 7 -12 2.29 2.06 .955 2.74 2.72 MVP-27,RoY-5
2004 21 FLA OF NL 158 158 135 1360.0 284 262 13 9 1 .968 -10 -3 -8 -2 1.82 1.74 .982 2.16 2.14 AS,MVP-22
2004 21 FLA RF NL 100 100 82 856.0 184 170 7 7 1 .962 -9 -9 -13 -12 1.86 1.77 .980 2.09 2.07 AS,MVP-22
2004 21 FLA LF NL 59 58 53 504.0 100 92 6 2 0 .980 0 6 0 14 1.75 1.66 .980 1.85 1.84 AS,MVP-22
2004 21 FLA DH NL 1 1 AS,MVP-22
2005 22 FLA LF NL 134 128 102 1105.2 205 188 12 5 3 .976 -9 -4 -10 -4 1.63 1.49 .984 1.94 1.92 AS,MVP-5,SS
2005 22 FLA OF NL 134 128 102 1105.2 205 188 12 5 3 .976 -9 -4 -10 -4 1.63 1.49 .984 2.19 2.16 AS,MVP-5,SS
2005 22 FLA 3B NL 29 28 20 238.0 70 22 4

In [73]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

path = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(path)


driver.get('https://www.baseball-reference.com/players/c/cabremi01.shtml')
print(driver.title)


search = driver.find_element_by_id('standard_fielding')
rows = search.find_elements_by_tag_name('tr')

for row in rows:
  print(row.text)
   

driver.quit()

<ipython-input-73-87f7872c7789>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Miguel Cabrera Stats | Baseball-Reference.com
Year Age Tm Pos Lg G GS CG Inn Ch PO A E DP Fld% Rtot Rdrs Rtot/yr Rdrs/yr RF/9 RF/G lgFld% lgRF9 lgRFG Awards
2003 20 FLA LF NL 55 55 51 481.0 107 99 5 3 1 .972 -5 1 -12 2 1.95 1.89 .979 1.91 1.89 MVP-27,RoY-5
2003 20 FLA OF NL 55 55 51 481.0 107 99 5 3 1 .972 -5 1 -12 2 1.95 1.89 .984 2.17 2.15 MVP-27,RoY-5


<ipython-input-73-87f7872c7789>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_id('standard_fielding')


2003 20 FLA 3B NL 34 30 30 275.0 71 17 53 1 2 .986 2 -3 7 -12 2.29 2.06 .955 2.74 2.72 MVP-27,RoY-5
2004 21 FLA OF NL 158 158 135 1360.0 284 262 13 9 1 .968 -10 -3 -8 -2 1.82 1.74 .982 2.16 2.14 AS,MVP-22
2004 21 FLA RF NL 100 100 82 856.0 184 170 7 7 1 .962 -9 -9 -13 -12 1.86 1.77 .980 2.09 2.07 AS,MVP-22
2004 21 FLA LF NL 59 58 53 504.0 100 92 6 2 0 .980 0 6 0 14 1.75 1.66 .980 1.85 1.84 AS,MVP-22
2004 21 FLA DH NL 1 1 AS,MVP-22
2005 22 FLA LF NL 134 128 102 1105.2 205 188 12 5 3 .976 -9 -4 -10 -4 1.63 1.49 .984 1.94 1.92 AS,MVP-5,SS
2005 22 FLA OF NL 134 128 102 1105.2 205 188 12 5 3 .976 -9 -4 -10 -4 1.63 1.49 .984 2.19 2.16 AS,MVP-5,SS
2005 22 FLA 3B NL 29 28 20 238.0 70 22 46 2 5 .971 -2 0 -8 0 2.57 2.34 .958 2.77 2.74 AS,MVP-5,SS
2006 23 FLA 3B NL 157 157 134 1334.0 397 114 266 17 33 .957 -6 -10 -5 -9 2.56 2.42 .954 2.70 2.68 AS,MVP-5,SS
2006 23 FLA DH NL 1 1 AS,MVP-5,SS
2007 24 FLA 3B NL 153 152 134 1310.2 389 100 266 23 33 .941 -10 -19 -9 -17 2.51 2.39 .954 2.68 2.67 AS,MVP-15